In [1]:
# Step 1: Load data set
import numpy as np
import sklearn
import pandas as pd

# 1. Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1.1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 1.2. Get the file
data_train = drive.CreateFile({'id':'1GWi5T9feYyYf2tfDdTaXZCKTtcrsQqga'}) # replace the id with id of file you want to access
data_train.GetContentFile('libsignal-protocol-c-data-v2.csv')  
df = pd.read_csv('libsignal-protocol-c-data-v2.csv') 

X_data = np.array(df.iloc[:, 0])
Y_data1 = np.array(df.iloc[:, 1])
Y_data2 = np.array(df.iloc[:, 2])
Y_data3 = np.array(df.iloc[:, 3])
Y_data4 = np.array(df.iloc[:, 4])
Y_data5 = np.array(df.iloc[:, 5])
#Y_data6 = np.array(df.iloc[:, 6])
#Y_data7 = np.array(df.iloc[:, 7])

df.head()

    100% |████████████████████████████████| 993kB 20.5MB/s 


,line,mem_access,signed_overflow,unsigned_overflow,shift,index_bound,division_by_zero,float_to_int
0,key unsigned modifier modifier type,0,0,0,0,0,0,0
1,key ident type,0,0,0,0,0,0,0
2,key type type,0,0,0,0,0,0,0
3,key unsigned type type,0,0,0,0,0,0,0
4,key unsigned type type,0,0,0,0,0,0,0


In [2]:
import seaborn as sns
unique, counts = np.unique(Y_data1, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data1, kde=False, rug=False)

unique, counts = np.unique(Y_data2, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data2, kde=False, rug=False)

unique, counts = np.unique(Y_data3, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data3, kde=False, rug=False)

unique, counts = np.unique(Y_data4, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data4, kde=False, rug=False)

unique, counts = np.unique(Y_data5, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data5, kde=False, rug=False)

{0: 38121, 1: 3368}
{0: 40314, 1: 1175}
{0: 40674, 1: 815}
{0: 40953, 1: 536}
{0: 41428, 1: 61}


In [3]:
# check if using GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# Step 2: Build word2vec model
import numpy as np
import sklearn
import pandas as pd

import gensim
import logging
import os

def tokenlizer(line):
  return [token for token in line.split(" ") if token != " "]

def read_input(input_data):
    for line in input_data:
        yield tokenlizer(line)
        
documents = list(read_input(X_data))

num_features = 30  # Word vector dimensionality 

# build vocabulary and train model
w2vmodel = gensim.models.Word2Vec(
    documents,
    size=num_features,
    window=10,
    min_count=2,
    workers=10)

w2vmodel.train(documents, total_examples=len(documents), epochs=10)

# save model to file
w2vmodel.wv.save_word2vec_format('model.bin', binary=True)

In [5]:
# Test word2vec model
# save only the word vectors
#model.wv.save(output_path)
word1 = "mul"
word1 in w2vmodel.wv.vocab
w2vmodel.wv.most_similar(positive = word1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ident', 0.49346357583999634),
 ('unsigned', 0.3912007212638855),
 ('open_round', 0.383032888174057),
 ('qualifier', 0.38014936447143555),
 ('type', 0.33830583095550537),
 ('access', 0.3282201588153839),
 ('comma', 0.31635046005249023),
 ('storage', 0.30941832065582275),
 ('bitwise_and', 0.3019465208053589),
 ('equal', 0.301226407289505)]

In [6]:
# Obvervation data using kmean
"""from nltk.cluster import KMeansClusterer
import nltk
import numpy as np 
  
from sklearn import cluster
from sklearn import metrics
from sklearn.decomposition import PCA
# #############################################################################
pca = PCA(n_components=2)
X = pca.fit_transform(X_vectors)
NUM_CLUSTERS=4
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=2, avoid_empty_clusters=True)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
#print (assigned_clusters)"""
  



'from nltk.cluster import KMeansClusterer\nimport nltk\nimport numpy as np \n  \nfrom sklearn import cluster\nfrom sklearn import metrics\nfrom sklearn.decomposition import PCA\n# #############################################################################\npca = PCA(n_components=2)\nX = pca.fit_transform(X_vectors)\nNUM_CLUSTERS=4\nkclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=2, avoid_empty_clusters=True)\nassigned_clusters = kclusterer.cluster(X, assign_clusters=True)\n#print (assigned_clusters)'

In [7]:
# Function to extract vector presentation for a line


def make_feature_vec(words, w2vmodel):
    
    nwords = 0

   #index2word_set = set(model.index2word)  # words known to the model
    #words = gensim.utils.simple_preprocess(line, deacc=False, min_len=1, max_len=100)
    feature_vec = np.zeros((num_features*len(words),),dtype="float32")  # pre-initialize (for speed)
    #print (" " + line)
    for word in words:
        nwords = nwords + 1.
        #feature_vec = np.add(feature_vec,w2vmodel.wv[word])
        feature_vec = np.concatenate((feature_vec,w2vmodel.wv[word]), axis=None)
    #if (nwords > 0):
       # feature_vec = np.divide(feature_vec, nwords)
    return feature_vec.T
  
def padding(X_vectors, max_lengh):
    features_matrix = [np.zeros((max_lengh,),dtype="float32") for _ in range(len(X_vectors))]
    for i in range (len(X_vectors)):
      feature_vec = np.empty(max_lengh)
      feature_vec[:len(X_vectors[i])] = X_vectors[i]
      feature_vec[len(X_vectors[i]):] = np.zeros((max_lengh - len(X_vectors[i]),),dtype="float32") 
      features_matrix[i] = feature_vec
    return features_matrix

def extract_features(X_data, w2vmodel):
    features_matrix = [np.zeros((num_features,),dtype="float32") for _ in range(len(X_data))]
    train_labels = np.zeros(len(X_data))
    
    max_lengh = 0
    for i in range (len(X_data)):
        if (len(X_data[i]) > 1):
            #if (i > 0):
             # words_pre = gensim.utils.simple_preprocess(X_data[i-1], deacc=False, min_len=1, max_len=100)
           # else:
             # words_pre = []
            words = gensim.utils.simple_preprocess(X_data[i], deacc=False, min_len=1, max_len=100)
          
            
            feature_vector =  make_feature_vec(words, w2vmodel)
            features_matrix[i] = feature_vector
            if max_lengh < len(feature_vector):
               max_lengh = len(feature_vector)
    
    return features_matrix, max_lengh
 

X_vectors, max_lengh = extract_features(X_data, w2vmodel)
print(max_lengh)

X_vectors_pad = padding(X_vectors, max_lengh)



2760


In [8]:
data_test = drive.CreateFile({'id':'16amsMuyPrx3Wo0HdotrjfgPQQZXaicJK'}) # replace the id with id of file you want to access
data_test.GetContentFile('c-compiler-data-v2.csv')  
df_test = pd.read_csv('c-compiler-data-v2.csv') 


X_data_test = np.array(df_test.iloc[:, 0])
Y_data1_test = np.array(df_test.iloc[:, 1])
Y_data2_test = np.array(df_test.iloc[:, 2])
Y_data3_test = np.array(df_test.iloc[:, 3])
Y_data4_test = np.array(df_test.iloc[:, 4])
Y_data5_test = np.array(df_test.iloc[:, 5])


X_vectors_test, max_lengh_test = extract_features(X_data_test, w2vmodel)
print(max_lengh_test)

X_vectors_pad_test = padding(X_vectors_test, max_lengh)

2580


In [9]:
import seaborn as sns
unique, counts = np.unique(Y_data1_test, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data1, kde=False, rug=False)

unique, counts = np.unique(Y_data2_test, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data2, kde=False, rug=False)

unique, counts = np.unique(Y_data3_test, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data3, kde=False, rug=False)

unique, counts = np.unique(Y_data4_test, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data4, kde=False, rug=False)

unique, counts = np.unique(Y_data5_test, return_counts=True)
print(dict(zip(unique, counts)))
#sns.distplot(Y_data5, kde=False, rug=False)

{0: 6679, 1: 1164}
{0: 7621, 1: 222}
{0: 7788, 1: 55}
{0: 7785, 1: 58}
{0: 7804, 1: 39}


In [10]:
# NB for mem_access
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

from sklearn.metrics import classification_report, accuracy_score, make_scorer

def classification_report_with_accuracy_score(y_true, y_pred):
    print (classification_report(y_true, y_pred) )
    return accuracy_score(y_true, y_pred)

nested_score = cross_val_score(clf,X_vectors_pad,Y_data1, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.93      0.86      0.90      9531
           1       0.15      0.28      0.20       842

   micro avg       0.82      0.82      0.82     10373
   macro avg       0.54      0.57      0.55     10373
weighted avg       0.87      0.82      0.84     10373

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      9530
           1       0.80      0.27      0.40       842

   micro avg       0.94      0.94      0.94     10372
   macro avg       0.87      0.63      0.68     10372
weighted avg       0.93      0.94      0.92     10372

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      9530
           1       0.19      0.15      0.17       842

   micro avg       0.88      0.88      0.88     10372
   macro avg       0.56      0.55      0.55     10372
weighted avg       0.87      0.88      0.87     10372

              preci

In [11]:
clf.fit(X_vectors_pad, Y_data1)
Y1_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data1_test, Y1_test_predic))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      6679
           1       0.54      0.28      0.37      1164

   micro avg       0.86      0.86      0.86      7843
   macro avg       0.71      0.62      0.64      7843
weighted avg       0.83      0.86      0.84      7843

0.8582175188065791


In [12]:
nested_score = cross_val_score(clf,X_vectors_pad,Y_data2, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.98      0.91      0.95     10079
           1       0.14      0.50      0.22       294

   micro avg       0.90      0.90      0.90     10373
   macro avg       0.56      0.71      0.59     10373
weighted avg       0.96      0.90      0.93     10373

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     10079
           1       0.78      0.16      0.27       294

   micro avg       0.97      0.97      0.97     10373
   macro avg       0.88      0.58      0.63     10373
weighted avg       0.97      0.97      0.97     10373

              precision    recall  f1-score   support

           0       0.98      0.97      0.97     10078
           1       0.23      0.33      0.27       294

   micro avg       0.95      0.95      0.95     10372
   macro avg       0.60      0.65      0.62     10372
weighted avg       0.96      0.95      0.95     10372

              preci

In [13]:
clf.fit(X_vectors_pad, Y_data2)
Y2_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data2_test, Y2_test_predic))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      7621
           1       0.19      0.30      0.24       222

   micro avg       0.94      0.94      0.94      7843
   macro avg       0.59      0.63      0.60      7843
weighted avg       0.96      0.94      0.95      7843

0.9446640316205533


In [14]:
nested_score = cross_val_score(clf,X_vectors_pad,Y_data3, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97     10169
           1       0.17      0.62      0.27       204

   micro avg       0.94      0.94      0.94     10373
   macro avg       0.58      0.78      0.62     10373
weighted avg       0.98      0.94      0.95     10373

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     10169
           1       0.69      0.54      0.61       204

   micro avg       0.99      0.99      0.99     10373
   macro avg       0.84      0.77      0.80     10373
weighted avg       0.99      0.99      0.99     10373

              precision    recall  f1-score   support

           0       0.98      0.95      0.96     10168
           1       0.04      0.10      0.06       204

   micro avg       0.93      0.93      0.93     10372
   macro avg       0.51      0.53      0.51     10372
weighted avg       0.96      0.93      0.95     10372

              preci

In [15]:
clf.fit(X_vectors_pad, Y_data3)
Y3_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data3_test, Y3_test_predic))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      7788
           1       0.07      0.51      0.12        55

   micro avg       0.95      0.95      0.95      7843
   macro avg       0.53      0.73      0.55      7843
weighted avg       0.99      0.95      0.97      7843

0.9495091164095372


In [16]:
nested_score = cross_val_score(clf,X_vectors_pad,Y_data4, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.99      0.92      0.95     10239
           1       0.08      0.57      0.14       134

   micro avg       0.91      0.91      0.91     10373
   macro avg       0.54      0.74      0.55     10373
weighted avg       0.98      0.91      0.94     10373

              precision    recall  f1-score   support

           0       1.00      0.84      0.91     10238
           1       0.06      0.70      0.10       134

   micro avg       0.84      0.84      0.84     10372
   macro avg       0.53      0.77      0.51     10372
weighted avg       0.98      0.84      0.90     10372

              precision    recall  f1-score   support

           0       1.00      0.77      0.87     10238
           1       0.05      1.00      0.10       134

   micro avg       0.77      0.77      0.77     10372
   macro avg       0.53      0.88      0.48     10372
weighted avg       0.99      0.77      0.86     10372

              preci

In [17]:
clf.fit(X_vectors_pad, Y_data4)
Y4_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data4_test, Y4_test_predic))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88      7785
           1       0.03      0.74      0.05        58

   micro avg       0.79      0.79      0.79      7843
   macro avg       0.51      0.77      0.47      7843
weighted avg       0.99      0.79      0.88      7843

0.7900038250669387


In [18]:
nested_score = cross_val_score(clf,X_vectors_pad,Y_data5, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       1.00      0.72      0.84     10357
           1       0.00      0.75      0.01        16

   micro avg       0.72      0.72      0.72     10373
   macro avg       0.50      0.74      0.42     10373
weighted avg       1.00      0.72      0.84     10373

              precision    recall  f1-score   support

           0       1.00      0.79      0.88     10357
           1       0.01      1.00      0.01        15

   micro avg       0.79      0.79      0.79     10372
   macro avg       0.50      0.89      0.45     10372
weighted avg       1.00      0.79      0.88     10372

              precision    recall  f1-score   support

           0       1.00      0.56      0.72     10357
           1       0.00      1.00      0.01        15

   micro avg       0.56      0.56      0.56     10372
   macro avg       0.50      0.78      0.36     10372
weighted avg       1.00      0.56      0.71     10372

              preci

In [19]:
clf.fit(X_vectors_pad, Y_data5)
Y5_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data5_test, Y5_test_predic))

              precision    recall  f1-score   support

           0       1.00      0.66      0.79      7804
           1       0.01      1.00      0.03        39

   micro avg       0.66      0.66      0.66      7843
   macro avg       0.51      0.83      0.41      7843
weighted avg       1.00      0.66      0.79      7843

0.656763993369884


In [20]:
from sklearn import svm

clf = svm.SVC(C=4, kernel= 'rbf', gamma='scale', decision_function_shape='ovo')

nested_score = cross_val_score(clf, X_vectors_pad,Y_data1, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      9531
           1       0.80      0.62      0.70       842

   micro avg       0.96      0.96      0.96     10373
   macro avg       0.89      0.80      0.84     10373
weighted avg       0.95      0.96      0.95     10373

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      9530
           1       0.96      0.69      0.81       842

   micro avg       0.97      0.97      0.97     10372
   macro avg       0.97      0.85      0.90     10372
weighted avg       0.97      0.97      0.97     10372

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9530
           1       0.92      0.93      0.92       842

   micro avg       0.99      0.99      0.99     10372
   macro avg       0.96      0.96      0.96     10372
weighted avg       0.99      0.99      0.99     10372

              preci

In [21]:
clf.fit(X_vectors_pad, Y_data1)
Y1_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data1_test, Y1_test_predic))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      6679
           1       0.88      0.82      0.85      1164

   micro avg       0.96      0.96      0.96      7843
   macro avg       0.92      0.90      0.91      7843
weighted avg       0.96      0.96      0.96      7843

0.9567767435930129


In [22]:
from sklearn import svm
clf = svm.SVC(C=4, kernel= 'rbf', gamma='scale', decision_function_shape='ovo')

# Nested CV with parameter optimization
nested_score = cross_val_score(clf, X_vectors_pad,Y_data2, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     10079
           1       0.64      0.72      0.68       294

   micro avg       0.98      0.98      0.98     10373
   macro avg       0.81      0.86      0.83     10373
weighted avg       0.98      0.98      0.98     10373

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     10079
           1       0.99      0.81      0.89       294

   micro avg       0.99      0.99      0.99     10373
   macro avg       0.99      0.91      0.95     10373
weighted avg       0.99      0.99      0.99     10373

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     10078
           1       0.73      0.73      0.73       294

   micro avg       0.98      0.98      0.98     10372
   macro avg       0.86      0.86      0.86     10372
weighted avg       0.98      0.98      0.98     10372

              preci

In [23]:
clf.fit(X_vectors_pad, Y_data2)
Y2_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data2_test, Y2_test_predic))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7621
           1       0.95      0.17      0.28       222

   micro avg       0.98      0.98      0.98      7843
   macro avg       0.96      0.58      0.64      7843
weighted avg       0.98      0.98      0.97      7843

0.9761570827489481


In [24]:
#from sklearn import svm
#clf = svm.SVC(C=4, kernel= 'rbf', gamma='scale', decision_function_shape='ovo')

# Nested CV with parameter optimization
nested_score = cross_val_score(clf, X_vectors_pad,Y_data3, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     10169
           1       0.66      1.00      0.79       204

   micro avg       0.99      0.99      0.99     10373
   macro avg       0.83      0.99      0.89     10373
weighted avg       0.99      0.99      0.99     10373

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10169
           1       0.93      0.88      0.90       204

   micro avg       1.00      1.00      1.00     10373
   macro avg       0.96      0.94      0.95     10373
weighted avg       1.00      1.00      1.00     10373

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     10168
           1       0.72      0.20      0.31       204

   micro avg       0.98      0.98      0.98     10372
   macro avg       0.85      0.60      0.65     10372
weighted avg       0.98      0.98      0.98     10372

              preci

In [25]:
clf.fit(X_vectors_pad, Y_data3)
Y3_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data3_test, Y3_test_predic))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7788
           1       0.44      0.93      0.60        55

   micro avg       0.99      0.99      0.99      7843
   macro avg       0.72      0.96      0.80      7843
weighted avg       1.00      0.99      0.99      7843

0.9913298482723447


In [26]:
#from sklearn import svm
#clf = svm.SVC(C=4, kernel= 'rbf', gamma='scale', decision_function_shape='ovo')

# Nested CV with parameter optimization
nested_score = cross_val_score(clf, X_vectors_pad,Y_data4, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10239
           1       0.97      0.67      0.79       134

   micro avg       1.00      1.00      1.00     10373
   macro avg       0.98      0.84      0.90     10373
weighted avg       1.00      1.00      1.00     10373

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10238
           1       1.00      0.62      0.76       134

   micro avg       1.00      1.00      1.00     10372
   macro avg       1.00      0.81      0.88     10372
weighted avg       1.00      1.00      0.99     10372

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10238
           1       0.98      1.00      0.99       134

   micro avg       1.00      1.00      1.00     10372
   macro avg       0.99      1.00      0.99     10372
weighted avg       1.00      1.00      1.00     10372

              preci

In [27]:
clf.fit(X_vectors_pad, Y_data4)
Y4_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data4_test, Y4_test_predic))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7785
           1       0.70      0.24      0.36        58

   micro avg       0.99      0.99      0.99      7843
   macro avg       0.85      0.62      0.68      7843
weighted avg       0.99      0.99      0.99      7843

0.9936248884355476


In [28]:
#from sklearn import svm
#clf = svm.SVC(C=4, kernel= 'rbf', gamma='scale', decision_function_shape='ovo')

# Nested CV with parameter optimization
nested_score = cross_val_score(clf, X_vectors_pad,Y_data5, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10357
           1       0.75      0.19      0.30        16

   micro avg       1.00      1.00      1.00     10373
   macro avg       0.87      0.59      0.65     10373
weighted avg       1.00      1.00      1.00     10373

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10357
           1       0.73      0.53      0.62        15

   micro avg       1.00      1.00      1.00     10372
   macro avg       0.86      0.77      0.81     10372
weighted avg       1.00      1.00      1.00     10372

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10357
           1       1.00      0.53      0.70        15

   micro avg       1.00      1.00      1.00     10372
   macro avg       1.00      0.77      0.85     10372
weighted avg       1.00      1.00      1.00     10372

              preci

In [29]:
clf.fit(X_vectors_pad, Y_data5)
Y5_test_predic = clf.predict(X_vectors_pad_test)
print(classification_report_with_accuracy_score(Y_data5_test, Y5_test_predic))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7804
           1       1.00      0.18      0.30        39

   micro avg       1.00      1.00      1.00      7843
   macro avg       1.00      0.59      0.65      7843
weighted avg       1.00      1.00      0.99      7843

0.9959199285987504


In [30]:
#Neural network
"""from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Conv1D, MaxPooling1D 
from keras.wrappers.scikit_learn import KerasClassifier 
import numpy as np 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

def create_model(): 

  model = Sequential()
  model.add(Conv1D(128, 2, padding='valid', activation='relu', strides=1, input_shape=(max_lengh,1))) 
  model.add(MaxPooling1D(2)) 
  model.add(Flatten()) 
  model.add(Dense(64, activation='relu')) 
  model.add(Dropout(0.5)) 
  model.add(Dense(32)) 
  model.add(Dense(1, activation='sigmoid')) 
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
clf = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

#x, y = generate_input('./data/vec')
x = np.reshape(X_vectors_pad, (-1, max_lengh, 1)) 
y = np.reshape(Y_data2, (-1, 1))

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
#model.fit(x_train, y_train, batch_size=32, epochs=1)

nested_score = cross_val_score(clf, x,y, cv=4, scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score) 
print("Accuracy: %0.2f (+/- %0.2f)" % (nested_score.mean(), nested_score.std() * 2))"""



'from keras.models import Sequential \nfrom keras.layers import Dense, Dropout, Activation, Flatten \nfrom keras.layers import Conv1D, MaxPooling1D \nfrom keras.wrappers.scikit_learn import KerasClassifier \nimport numpy as np \nfrom sklearn.metrics import accuracy_score \nfrom sklearn.model_selection import train_test_split\n\ndef create_model(): \n\n  model = Sequential()\n  model.add(Conv1D(128, 2, padding=\'valid\', activation=\'relu\', strides=1, input_shape=(max_lengh,1))) \n  model.add(MaxPooling1D(2)) \n  model.add(Flatten()) \n  model.add(Dense(64, activation=\'relu\')) \n  model.add(Dropout(0.5)) \n  model.add(Dense(32)) \n  model.add(Dense(1, activation=\'sigmoid\')) \n  model.compile(loss=\'binary_crossentropy\', optimizer=\'adam\', metrics=[\'accuracy\'])\n  return model\nclf = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)\n\n#x, y = generate_input(\'./data/vec\')\nx = np.reshape(X_vectors_pad, (-1, max_lengh, 1)) \ny = np.reshape(Y_data2, (-

In [31]:
"""model = Sequential() 
  model.add(Conv1D(64, kernel_size=(1, 2), activation='relu', input_shape=(max_lengh,1))) 
  model.add(Conv2D(128, (1, 2), activation='relu'))
  model.add(MaxPooling1D(pool_size=(2))) 
  model.add(Dropout(0.25)) 
  model.add(Flatten()) 
  model.add(Dense(128, activation='relu')) 
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax')) 
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])"""

"model = Sequential() \n  model.add(Conv1D(64, kernel_size=(1, 2), activation='relu', input_shape=(max_lengh,1))) \n  model.add(Conv2D(128, (1, 2), activation='relu'))\n  model.add(MaxPooling1D(pool_size=(2))) \n  model.add(Dropout(0.25)) \n  model.add(Flatten()) \n  model.add(Dense(128, activation='relu')) \n  model.add(Dropout(0.5))\n  model.add(Dense(num_classes, activation='softmax')) \n  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])"